# LangGraph와 AgentCore Memory Tool (단기 메모리)

## 소개
이 노트북은 LangGraph 프레임워크를 사용하여 Amazon Bedrock AgentCore Memory 기능을 대화형 AI 에이전트와 통합하는 방법을 보여줍니다. 단일 대화 세션 내에서 **단기 메모리** 보존에 초점을 맞추어 에이전트가 명시적인 컨텍스트 관리 없이 대화 초기의 정보를 기억할 수 있도록 합니다.


## 튜토리얼 세부사항

| 정보                | 세부사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형        | 단기 대화형                                                                        |
| 에이전트 사용 사례     | 개인 피트니스                                                                     |
| 에이전트 프레임워크   | Langgraph                                                                        |
| LLM 모델            | Anthropic Claude Sonnet 3.7                                                      |
| 튜토리얼 구성요소     | AgentCore 단기 메모리, Langgraph, 도구를 통한 메모리 검색                |
| 예제 복잡도          | 초급                                                                             |

배울 내용:
- 단기 메모리를 위한 AgentCore Memory로 메모리 저장소 생성
- LangGraph를 사용하여 구조화된 메모리 워크플로우를 가진 에이전트 생성
- 대화 기록 검색을 위한 메모리 도구 구현
- 단일 세션 내에서 컨텍스트 정보 액세스 및 활용
- 효과적인 메모리 회상을 통한 대화 경험 향상


### 시나리오 컨텍스트

이 예제에서는 대화 전반에 걸쳐 언급되는 운동 세부사항, 피트니스 목표, 신체적 제약, 운동 선호도를 기억할 수 있는 "**개인 피트니스 코치**"를 만들어보겠습니다. 이 어시스턴트는 효과적인 단기 메모리 관리가 사용자가 정보를 반복적으로 말할 필요 없이 더 자연스럽고 개인화된 피트니스 코칭 경험을 가능하게 하는 방법을 보여줍니다.


## 아키텍처
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 전제조건

- Python 3.10+
- 적절한 권한이 있는 AWS 계정
- AgentCore Memory에 대한 적절한 권한이 있는 AWS IAM 역할
- Amazon Bedrock 모델에 대한 액세스

환경 설정을 시작해보겠습니다!

## 1단계: 환경 설정
이 노트북이 작동하도록 필요한 모든 라이브러리를 가져오고 클라이언트를 정의하는 것부터 시작해보겠습니다.

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import logging
from datetime import datetime

Amazon Bedrock 모델과 AgentCore에 대한 적절한 권한을 가진 지역과 역할을 정의합니다

In [ ]:
import os
region = os.getenv('AWS_REGION', 'us-west-2')

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("agentcore-memory")

### 통합 작동 방식

LangGraph와 AgentCore Memory 간의 통합은 다음을 포함합니다:

1. AgentCore Memory를 사용하여 단기 메모리에 대화 저장
2. LangGraph의 구조화된 워크플로우로 메모리 작업 관리

이 접근 방식은 메모리 관리를 추론에서 분리하여 더 깨끗하고 유지보수가 가능한 에이전트 아키텍처를 만들어줍니다.

## 2단계: 메모리 생성
이 섹션에서는 AgentCore Memory SDK를 사용하여 메모리 저장소를 생성합니다. 이 메모리 저장소는 에이전트가 대화에서 정보를 보유할 수 있게 해줍니다.

In [ ]:
from bedrock_agentcore.memory import MemoryClient
from botocore.exceptions import ClientError

In [ ]:
client = MemoryClient(region_name=region)
memory_name = "FitnessCoach"
memory_id = None

In [ ]:
try:
    print("Creating Memory...")
    # Create the memory resource
    memory = client.create_memory_and_wait(
        name=memory_name,                       # This name is unique across all memories in this account
        description="Fitness Coach Agent",      # Human-readable description
        strategies=[],                          # No memory strategies for short-term memory
        event_expiry_days=7,                    # Memories expire after 7 days
        max_wait=300,                           # Maximum time to wait for memory creation (5 minutes)
        poll_interval=10                        # Check status every 10 seconds
    )

    # Extract and print the memory ID
    memory_id = memory['id']
    logger.info(f"Memory created successfully with ID: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    logger.info(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

## 3단계: LangGraph 에이전트 생성
LangGraph로 에이전트를 생성하는 데 필요한 모든 라이브러리를 가져오겠습니다.

In [ ]:
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_aws import ChatBedrock

### LangGraph 에이전트 구현

이제 메모리 도구를 통합하여 LangGraph로 에이전트를 생성해보겠습니다:

In [ ]:
def create_agent(client, memory_id, actor_id, session_id):
    """Create and configure the LangGraph agent"""
    
    # Initialize your LLM (adjust model and parameters as needed)
    llm = ChatBedrock(
        model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # or your preferred model
        model_kwargs={"temperature": 0.1}
    )
    
    @tool
    def list_events():
        """Tool used when needed to retrieve recent information""" 
        events = client.list_events(
                memory_id=memory_id,
                actor_id=actor_id,
                session_id=session_id,
                max_results=10
            )
        return events
        
    
    # Bind tools to the LLM
    tools = [list_events]
    llm_with_tools = llm.bind_tools(tools)
    
    # System message
    system_message = """You are the Personal Fitness Coach, a sophisticated fitness guidance assistant.
                        PURPOSE:
                        - Help users develop workout routines based on their fitness goals
                        - Remember user's exercise preferences, limitations, and progress
                        - Provide personalized fitness recommendations and training plans
                        MEMORY CAPABILITIES:
                        - You have access to recent events with the list_events tool
                        """
    
    # Define the chatbot node
    def chatbot(state: MessagesState):
        raw_messages = state["messages"]
    
        # Remove any existing system messages to avoid duplicates or misplacement
        non_system_messages = [msg for msg in raw_messages if not isinstance(msg, SystemMessage)]
    
        # Always ensure SystemMessage is first
        messages = [SystemMessage(content=system_message)] + non_system_messages
    
        latest_user_message = next((msg.content for msg in reversed(messages) if isinstance(msg, HumanMessage)), None)
    
        # Get response from model with tools bound
        response = llm_with_tools.invoke(messages)
    
        # Save conversation if applicable
        if latest_user_message and response.content.strip():  # Check that response has content
            conversation = [
                (latest_user_message, "USER"),
                (response.content, "ASSISTANT")
            ]
            
            # Validate that all message texts are non-empty
            if all(msg[0].strip() for msg in conversation):  # Ensure no empty messages
                try:
                    client.create_event(
                        memory_id=memory_id,
                        actor_id=actor_id,
                        session_id=session_id,
                        messages=conversation
                    )
                except Exception as e:
                    print(f"Error saving conversation: {str(e)}")
        
        # Append response to full message history
        return {"messages": raw_messages + [response]}
    
    # Create the graph
    graph_builder = StateGraph(MessagesState)
    
    # Add nodes
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # Add edges
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # Set entry point
    graph_builder.set_entry_point("chatbot")
    
    # Compile the graph
    return graph_builder.compile()

### 에이전트 호출을 위한 래퍼 생성

에이전트를 호출하기 위한 간단한 래퍼를 만들어보겠습니다:

In [ ]:
def langgraph_bedrock(payload, agent):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    
    # Create the input in the format expected by LangGraph
    response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # Extract the final message content
    return response["messages"][-1].content

## 4단계: LangGraph 에이전트 실행
이제 AgentCore Memory 통합으로 에이전트를 실행할 수 있습니다.

In [ ]:
# Create unique actor and session IDs for this conversation
actor_id = f"user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
session_id = f"workout-{datetime.now().strftime('%Y%m%d%H%M%S')}"

In [ ]:
# Create the agent with AgentCore Memory integration
agent = create_agent(client, memory_id, actor_id, session_id)

#### 축하합니다! 에이전트가 준비되었습니다!!

### 에이전트를 테스트해보겠습니다

에이전트의 메모리 기능을 테스트하기 위해 에이전트와 상호작용해보겠습니다:

In [ ]:
response = langgraph_bedrock({"prompt": "Hello! This is my first day, I need a workout routine."}, agent)
print(f"Agent: {response}\n")

In [ ]:
response = langgraph_bedrock({"prompt": "I want to build muscle, looking for a biceps routine. I have some lower back problems."}, agent)
print(f"Agent: {response}\n")

In [ ]:
response = langgraph_bedrock({"prompt": "Can you give me three exercises with number of reps?"}, agent)
print(f"Agent: {response}\n")

### 메모리 지속성 테스트

AgentCore Memory 통합의 위력을 진정으로 보여주기 위해 새로운 에이전트 인스턴스를 생성하고 이전 대화를 기억할 수 있는지 확인해보겠습니다:

In [ ]:
# Create a new agent instance (simulating a new session)
new_agent = create_agent(client, memory_id, actor_id, session_id)

# Test if the new agent remembers our preferences
response = langgraph_bedrock({
    "prompt": "Hello again! Can you remind me about my last workout session?"
}, new_agent)

print("New Agent Session:\n")
print(f"Agent: {response}")

## 요약

이 노트북에서 보여준 내용:

1. AI 에이전트를 위한 AgentCore Memory 리소스 생성 방법
2. 메모리 통합을 가진 LangGraph 워크플로우 구축
3. 대화 기록 검색을 위한 메모리 도구 구현
4. 필요할 때 지능적으로 메모리를 사용하는 에이전트 생성
5. 에이전트 인스턴스 간 메모리 지속성 테스트

이 통합은 구조화된 워크플로우(LangGraph)와 강력한 메모리 시스템(AgentCore Memory)을 결합하여 더 지능적이고 컨텍스트를 인식하는 AI 에이전트를 만드는 위력을 보여줍니다.

우리가 보여준 접근 방식은 멀티 에이전트 시스템, 추출 전략을 가진 장기 메모리, 대화 컨텍스트에 기반한 전문 메모리 검색 등 더 복잡한 사용 사례로 확장할 수 있습니다.

## 정리
이 노트북에서 사용한 리소스를 정리하기 위해 메모리를 삭제해보겠습니다.

In [ ]:
#client.delete_memory_and_wait(memory_id = memory_id, max_wait = 300, poll_interval =10)